## Recovering Correlation Energy: B atom

:::{panels} 
:column: col-12
Based on **Basis sets**
 Tricia D. Shepherd [![Orcid](../../images/orcid.png)](https://orcid.org/0000-0001-6512-8951), Ryan C. Fortenberry, Matthewy Kennedy, C. David Sheril, 2008-2019, The Psi4Education Developers &nbsp;&nbsp; License BSD-3
:::

At the Hartree Fock level of theory, each electron experiences an average potential field of all the other electrons. In essence, it is a "mean field" approach that neglects individual electron-electron interactions or "electron correlation". Thus, we define the difference between the self-consistent field energy and the exact energy as the correlation energy. Two fundamentally different approaches to account for electron correlation effects are available by selecting a Correlation method: Moller Plesset (MP) Perturbation theory and Coupled Cluster (CC) theory. 

:::{admonition} Exercise 1
:class: exercise 

* Calculate the HF energy for Boron using the 6-311+G** basis set, then determine the value of the correlation energy for boron assuming an "experimental" energy of -24.608 Hartrees ([Schaefer III, Henry F., and Frank E. Harris. "Electronic Structure of Atomic Boron." Physical Review 167.1 (1968): 67](https://journals.aps.org/pr/abstract/10.1103/PhysRev.167.67)).  
* Using the same basis set, perform an energy calculation with CISD and full CI. 
* Using the same basis set, perform an energy calculation with MP2, MP3 and MP5. 
* Using the same basis set, perform an energy calculation with CCSD and CCSD(T).

Determine the percentage of the correlation energy recovered for HF, MP2, MP5, CCSD, CCSD(T). 

Compare the performances of the different methods. In particular, you will see that some of the methods seem to overestimate the correlation recovered. How is it possible? (*Hint:* Think both about the theoretical aspects of the methods and *how* we are computing the recovered correlation energy).
:::

:::{admonition} CCSD energy/mp2 energy
:class: tip
You may recover the CCSD energy from the CCSD(T) calculation, but you will have to look at the output file or by calculating both. In any case, the calculations for a single atom are quick.  

The same is true for the MP2 energy which can be extracted from the MP5 calculation.
MP5 will require the use of the following options: `psi4.set_options({"reference" :"rohf", "qc_module":"detci"})` as Psi4 does not have MP5 implemented using a RHF reference. 
:::

In [1]:
import psi4
import py3Dmol
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append("..")
from helpers import *

In [2]:
psi4.set_num_threads(2)
psi4.set_memory('2 GB')

  Threads set to 2 by Python driver.

  Memory set to   1.863 GiB by Python driver.


2000000000

In [3]:
psi4.core.clean_options()

# define a single boron atom and set the correct multiplicity
boron_geometry = psi4.geometry("""
0 2
B 0.0 0.0 0.0
""")

psi4.set_options({'reference' : "UHF"})

# Perform single-point calculation with requested method/basis 
B_energy_hf  = psi4.energy(f'hf/{'6-311+G**'}', molecule=boron_geometry)


Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on noto.epfl.ch
*** at Fri Oct 10 22:50:26 2025

   => Loading Basis Set <=

    Name: 6-311+G**
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry B          line   108 file /opt/CompChem/psi4conda-1.9.1/share/psi4/basis/6-311pgss.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              UHF Reference
                        2 Threads,   1907 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: d2h
    Geometry (in Angstrom), charge = 0, multiplicity = 2:

      

In [4]:
drawXYZ(boron_geometry)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
#Experimental energy of Boron
B_exact_e = -24.608

correlation = B_exact_e - B_energy_hf # Calculate the total amount of correlation energy that should be recovered using the B_exact_e

print(F"\n Electron correlation: {correlation} Hartrees")


 Electron correlation: -0.07765482722757966 Hartrees


In [6]:
# calculate energy for other methods
psi4.core.clean_options()
psi4.core.clean()
psi4.set_options({'reference' : "UHF"})
#MP2
e_mp2 = None
#MP3
e_mp3 = None
#CCSD
e_ccsd = None
#CCSD(T)
e_ccsd_t = None
#MP5
psi4.core.clean_options()
psi4.core.clean()
psi4.set_options({"reference" :"rohf", 
                   "qc_module":"detci"})
e_mp5 = None
#CISD
e_cisd = None
#FCI
e_fci = None

In [7]:
# calculate energy for other methods
psi4.core.clean_options()
psi4.core.clean()

# --- Coupled-cluster & low-order MP with UHF reference ---
psi4.set_options({'reference': 'UHF'})

# MP2 / MP3
e_mp2   = psi4.energy('mp2/6-311+G**',molecule=boron_geometry)
e_mp3   = psi4.energy('mp3/6-311+G**',molecule=boron_geometry)

# CCSD and CCSD(T)
# (with reference='UHF' Psi4 runs the unrestricted variants under the hood)
e_ccsd   = psi4.energy('ccsd/6-311+G**',molecule=boron_geometry)
e_ccsd_t = psi4.energy('ccsd(t)/6-311+G**',molecule=boron_geometry)   # you could also read CCSD from this output

# --- High-order MP and CI with ROHF + detci (as required by Psi4 for MP5) ---
psi4.core.clean_options()
psi4.core.clean()
psi4.set_options({'reference': 'rohf', 'qc_module': 'detci'})

# MP5 (via detci)
e_mp5  = psi4.energy('mp5/6-311+G**',molecule=boron_geometry)

# CI
e_cisd = psi4.energy('cisd/6-311+G**',molecule=boron_geometry)
e_fci  = psi4.energy('fci/6-311+G**',molecule=boron_geometry)





Scratch directory: /tmp/
    SCF Algorithm Type (re)set to DF.
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on noto.epfl.ch
*** at Fri Oct 10 22:50:26 2025

   => Loading Basis Set <=

    Name: 6-311+G**
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry B          line   108 file /opt/CompChem/psi4conda-1.9.1/share/psi4/basis/6-311pgss.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              UHF Reference
                        2 Threads,   1907 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: d2h
    Geometry (in Angstrom),

In [8]:
# Print energies for verification
print(f"MP2 Energy: {e_mp2} Hartrees")
print(f"MP3 Energy: {e_mp3} Hartrees")
print(f"CCSD Energy: {e_ccsd} Hartrees")
print(f"CCSD(T) Energy: {e_ccsd_t} Hartrees")
print(f"MP5 Energy: {e_mp5} Hartrees")
print(f"CISD Energy: {e_cisd} Hartrees")
print(f"FCI Energy: {e_fci} Hartrees")

MP2 Energy: -24.586289575121743 Hartrees
MP3 Energy: -24.6008013484203 Hartrees
CCSD Energy: -24.60855241969002 Hartrees
CCSD(T) Energy: -24.609669719516056 Hartrees
MP5 Energy: -24.60860981858081 Hartrees
CISD Energy: -24.60742517905756 Hartrees
FCI Energy: -24.61004397508874 Hartrees


In [9]:
correlations = {}
correlations['hf']= 0
correlations['mp2']    = None # calculate the % of correlation energy recovered here
correlations['mp3']    = None # calculate the % of correlation energy recovered here
correlations['mp5']    = None # calculate the % of correlation energy recovered here
correlations['ccsd']   = None # calculate the % of correlation energy recovered here
correlations['ccsd(t)'] = None # calculate the % of correlation energy recovered here
correlations['cisd'] = None # calculate the % of correlation energy recovered here
correlations['fci'] = None # calculate the % of correlation energy recovered here

In [10]:
#Experimental energy of Boron
B_exact_e = -24.608

correlation = B_exact_e - B_energy_hf 

def corr_pct(e_method):
    return 100.0 * (e_method - B_energy_hf) / correlation

correlations = {}
correlations['hf']      = 0.0
correlations['mp2']     = corr_pct(e_mp2)
correlations['mp3']     = corr_pct(e_mp3)
correlations['mp5']     = corr_pct(e_mp5)
correlations['ccsd']    = corr_pct(e_ccsd)
correlations['ccsd(t)'] = corr_pct(e_ccsd_t)
correlations['cisd']    = corr_pct(e_cisd)
correlations['fci']     = corr_pct(e_fci)   # may be >100% with a finite basis

correlations


{'hf': 0.0,
 'mp2': 72.04240141487692,
 'mp3': 90.72993677701122,
 'mp5': 100.78529384789184,
 'ccsd': 100.71137842905947,
 'ccsd(t)': 102.15018122590364,
 'cisd': 99.25977435922296,
 'fci': 102.63212882149479}

In [11]:
pd.DataFrame.from_dict(correlations,orient='index',columns=['% correlation recovered'])

,% correlation recovered
hf,0.000000
mp2,72.042401
mp3,90.729937
mp5,100.785294
ccsd,100.711378
ccsd(t),102.150181
cisd,99.259774
fci,102.632129
